# Language Warmup Full Model

## Data Cleaning

## Feature Engineering and Vectorization

In [29]:
import random
import numpy as np

with open("FeatureCreate\words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]

phrases = []
for i in range (0,100):
    phraselength = random.randint(5,15)
    phrase = []
    for j in range(0,phraselength):
        choice = random.randint(0,len(words))
        phrase.append(words[choice])
        sentence = ' '.join(phrase)
    phrases.append(sentence)

    
print(len(phrases))
# REMOVE ABOVE

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True, lowercase=False, ngram_range=(2, 2))
vector = vectorizer.fit_transform(phrases)


100


In [24]:
# Make fake y values

y_val = [0] * 100
for i in range(0,50):
    y_val[i] = 1
        
print(y_val)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [35]:
data = vector.todense()
data = np.asarray(data)
print(type(data))

<class 'numpy.ndarray'>


In [40]:
x_train = np.concatenate([data[:40], data[:-40]])
print(x_train[1].size)

928


## Model Architecture

In [5]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(16, activation = 'relu'))
model.add(layers.Dense(1,  activation = 'sigmoid'))

Using TensorFlow backend.


In [6]:
model.compile(optimizer = 'rmsprop', 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])